<a href="https://colab.research.google.com/github/cbi-automation/lk-extraction/blob/fitriadc-dev/Main_effective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
!pip install PyMuPDF

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings, logging, re, os
!pip install pdfplumber
import pdfplumber
import pandas as pd
from google.colab import files
import time
# Impor file yang diunduh

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Catat waktu mulai
start_program_time = time.time()

!wget https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py -O marker_config.py

# Path dasar
base_path = "/content/drive/MyDrive/IDX_CALK_FX_SB_82"
output_path = os.path.join(base_path, "output")
os.makedirs(output_path, exist_ok=True)

# Daftar perusahaan yang ingin diproses

# Daftar perusahaan
all_companies = [
    "TLKM", "INDF", "INKP", "ADRO", "UNTR", "JSMR", "ICBP", "MEDC", "ISAT", "GIAA",
    "PGAS", "EXCL", "TPIA", "SMGR", "MDKA", "BHIT", "BSDE", "WIKA", "IMAS", "TKIM",
    "HMSP", "LPKR", "INDY", "BYAN", "INCO", "TBIG", "CTRA", "FREN", "EMTK", "CPIN",
    "ANTM", "SMAR", "PTBA", "ADHI", "PANI", "HRUM", "AMRT", "BMTR", "SIMP", "JPFA",
    "ITMG", "PWON", "ABMM", "AKRA", "AALI", "INTP", "KLBF", "MAPI", "BIPI", "APLN",
    "MYOR", "DOID", "TBLA", "BUKA", "MNCN", "ENRG", "ASRI", "CMNP", "SMCB", "DNET",
    "AUTO", "BKSL", "CENT", "GEMS", "SMDR", "GJTL", "UNVR", "BRMS", "CMNT", "KAEF",
    "DSNG", "DUTI", "TOBA", "DILD", "TAPG", "MDLN", "NIRO", "JRPT", "PSAB", "KIJA",
    "TINS", "LSIP", "ARCI", "MLPL", "FASW", "INDR", "PLIN", "MCOL", "TSPC", "SSMS",
    "CSAP", "MAPA", "PTRO", "EPMT", "SILO", "SCMA", "LPCK", "MTDL", "ELSA", "SGRO"
]

# Tahun dan kuartal yang ingin diproses
years = ["2023", "2024"]
quarters = ["Q1", "Q2", "Q3"]
max_workers = min(10, os.cpu_count() or 4)
batch_size = 10

import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)


--2025-05-05 06:57:39--  https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23534 (23K) [text/plain]
Saving to: ‘marker_config.py’

marker_config.py    100%[===================>]  22.98K  --.-KB/s    in 0.002s  

2025-05-05 06:57:39 (12.6 MB/s) - ‘marker_config.py’ saved [23534/23534]



In [ ]:
from marker_config import (
    ELSA_marker_config,
    SGRO_marker_config,
    TLKM_marker_config,
    INDF_marker_config,
    INKP_marker_config,
    ADRO_marker_config,
    UNTR_marker_config,
    JSMR_marker_config,
    ICBP_marker_config,
    MEDC_marker_config,
    ISAT_marker_config,
    GIAA_marker_config,
    PGAS_marker_config,
    EXCL_marker_config,
    TPIA_marker_config,
    SMGR_marker_config,
    MDKA_marker_config,
    TOWR_marker_config,
    BHIT_marker_config,
    BSDE_marker_config,
    WIKA_marker_config,
    IMAS_marker_config,
    TKIM_marker_config,
    PTPP_marker_config,
    HMSP_marker_config,
    LPKR_marker_config,
    DSSA_marker_config,
    INDY_marker_config,
    BYAN_marker_config,
    INCO_marker_config,
    TBIG_marker_config,
    CTRA_marker_config,
    FREN_marker_config,
    EMTK_marker_config,
    KRAS_marker_config,
    CPIN_marker_config,
    ANTM_marker_config,
    SMAR_marker_config,
    PTBA_marker_config,
    ADHI_marker_config,
    PANI_marker_config,
    HRUM_marker_config,
    AMRT_marker_config,
    BMTR_marker_config,
    SIMP_marker_config,
    JPFA_marker_config,
    ITMG_marker_config,
    KPIG_marker_config,
    PWON_marker_config,
    ABMM_marker_config,
    SMRA_marker_config,
    AKRA_marker_config,
    AALI_marker_config,
    INTP_marker_config,
    KLBF_marker_config,
    MAPI_marker_config,
    BIPI_marker_config,
    APLN_marker_config,
    MYOR_marker_config,
    DOID_marker_config,
    TBLA_marker_config,
    BUKA_marker_config,
    MNCN_marker_config,
    ERAA_marker_config,
    ENRG_marker_config,
    ASRI_marker_config,
    CMNP_marker_config,
    SMCB_marker_config,
    POWR_marker_config,
    DNET_marker_config,
    AUTO_marker_config,
    BKSL_marker_config,
    CENT_marker_config,
    GEMS_marker_config,
    SMDR_marker_config,
    GJTL_marker_config,
    PPRO_marker_config,
    # GGRP_marker_config,
    UNVR_marker_config,
    BRMS_marker_config,
    CMNT_marker_config,
    KAEF_marker_config,
    DSNG_marker_config,
    DUTI_marker_config,
    TOBA_marker_config,
    DILD_marker_config,
    TAPG_marker_config,
    MDLN_marker_config,
    NIRO_marker_config,
    JRPT_marker_config,
    PSAB_marker_config,
    KIJA_marker_config,
    TINS_marker_config,
    LSIP_marker_config,
    ARCI_marker_config,
    MLPL_marker_config,
    FASW_marker_config,
    INDR_marker_config,
    PLIN_marker_config,
    MCOL_marker_config,
    TSPC_marker_config,
    SSMS_marker_config,
    CSAP_marker_config,
    MAPA_marker_config,
    PTRO_marker_config,
    EPMT_marker_config,
    SILO_marker_config,
    SCMA_marker_config,
    LPCK_marker_config,
    MTDL_marker_config
)

marker_config_map = {
    "ELSA":ELSA_marker_config,
    "SGRO": SGRO_marker_config,
    "TLKM": TLKM_marker_config,
    "INDF": INDF_marker_config,
    "INKP": INKP_marker_config,
    "ADRO": ADRO_marker_config,
    "UNTR": UNTR_marker_config,
    "JSMR": JSMR_marker_config,
    "ICBP": ICBP_marker_config,
    "MEDC": MEDC_marker_config,
    "ISAT": ISAT_marker_config,
    "GIAA": GIAA_marker_config,
    "PGAS": PGAS_marker_config,
    "EXCL": EXCL_marker_config,
    "TPIA": TPIA_marker_config,
    "SMGR": SMGR_marker_config,
    "MDKA": MDKA_marker_config,
    "TOWR": TOWR_marker_config,
    "BHIT": BHIT_marker_config,
    "BSDE": BSDE_marker_config,
    "WIKA": WIKA_marker_config,
    "IMAS": IMAS_marker_config,
    "TKIM": TKIM_marker_config,
    "PTPP": PTPP_marker_config,
    "HMSP": HMSP_marker_config,
    "LPKR": LPKR_marker_config,
    "DSSA": DSSA_marker_config,
    "INDY": INDY_marker_config,
    "BYAN": BYAN_marker_config,
    "INCO": INCO_marker_config,
    "TBIG": TBIG_marker_config,
    "CTRA": CTRA_marker_config,
    "FREN": FREN_marker_config,
    "EMTK": EMTK_marker_config,
    "KRAS": KRAS_marker_config,
    "CPIN": CPIN_marker_config,
    "ANTM": ANTM_marker_config,
    "SMAR": SMAR_marker_config,
    "PTBA": PTBA_marker_config,
    "ADHI": ADHI_marker_config,
    "PANI": PANI_marker_config,
    "HRUM": HRUM_marker_config,
    "AMRT": AMRT_marker_config,
    "BMTR": BMTR_marker_config,
    "SIMP": SIMP_marker_config,
    "JPFA": JPFA_marker_config,
    "ITMG": ITMG_marker_config,
    "KPIG": KPIG_marker_config,
    "PWON": PWON_marker_config,
    "ABMM": ABMM_marker_config,
    "SMRA": SMRA_marker_config,
    "AKRA": AKRA_marker_config,
    "AALI": AALI_marker_config,
    "INTP": INTP_marker_config,
    "KLBF": KLBF_marker_config,
    "MAPI": MAPI_marker_config,
    "BIPI": BIPI_marker_config,
    "APLN": APLN_marker_config,
    "MYOR": MYOR_marker_config,
    "DOID": DOID_marker_config,
    "TBLA": TBLA_marker_config,
    "BUKA": BUKA_marker_config,
    "MNCN": MNCN_marker_config,
    "ERAA": ERAA_marker_config,
    "ENRG": ENRG_marker_config,
    "ASRI": ASRI_marker_config,
    "CMNP": CMNP_marker_config,
    "SMCB": SMCB_marker_config,
    "POWR": POWR_marker_config,
    "DNET": DNET_marker_config,
    "AUTO": AUTO_marker_config,
    "BKSL": BKSL_marker_config,
    "CENT": CENT_marker_config,
    "GEMS": GEMS_marker_config,
    "SMDR": SMDR_marker_config,
    "GJTL": GJTL_marker_config,
    "PPRO": PPRO_marker_config,
    # "GGRP": GGRP_marker_config,
    "UNVR": UNVR_marker_config,
    "BRMS": BRMS_marker_config,
    "CMNT": CMNT_marker_config,
    "KAEF": KAEF_marker_config,
    "DSNG": DSNG_marker_config,
    "DUTI": DUTI_marker_config,
    "TOBA": TOBA_marker_config,
    "DILD": DILD_marker_config,
    "TAPG": TAPG_marker_config,
    "MDLN": MDLN_marker_config,
    "NIRO": NIRO_marker_config,
    "JRPT": JRPT_marker_config,
    "PSAB": PSAB_marker_config,
    "KIJA": KIJA_marker_config,
    "TINS": TINS_marker_config,
    "LSIP": LSIP_marker_config,
    "ARCI": ARCI_marker_config,
    "MLPL": MLPL_marker_config,
    "FASW": FASW_marker_config,
    "INDR": INDR_marker_config,
    "PLIN": PLIN_marker_config,
    "MCOL": MCOL_marker_config,
    "TSPC": TSPC_marker_config,
    "SSMS": SSMS_marker_config,
    "CSAP": CSAP_marker_config,
    "MAPA": MAPA_marker_config,
    "PTRO": PTRO_marker_config,
    "EPMT": EPMT_marker_config,
    "SILO": SILO_marker_config,
    "SCMA": SCMA_marker_config,
    "LPCK": LPCK_marker_config,
    "MTDL": MTDL_marker_config
}

In [ ]:
import fitz  # PyMuPDF
import os
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# Fungsi ekstraksi teks yang cepat
def extract_text(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text


In [ ]:
from dataclasses import dataclass, field
from typing import Dict, Optional

@dataclass
class Company:
    kuartal: str = "-"
    perusahaan: str = "-"
    satuan: str = "-"
    risiko_nilai_tukar: str = "-"


# Mapping: atribut class -> nama key di dictionary input
field_map: Dict[str, str] = {
    "kuartal": "Kuartal",
    "perusahaan": "Perusahaan",
    "satuan": "Disajikan dalam",
    "risiko_nilai_tukar": "Efek Risiko Nilai Tukar",
    "risiko_suku_bunga": "Efek Risiko Suku Bunga",
}

def get_str(value) -> str:
    if value is None:
        return "-"
    val = str(value).strip()
    return val if val else "-"

def generate_company(doc: Optional[dict]) -> Company:
    data = {}
    for attr, source_key in field_map.items():
        data[attr] = get_str(doc.get(source_key)) if doc else "-"
    return Company(**data)

import json

company = Company()

In [ ]:

def find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal="2022"):
    start_marker, end_marker = marker_pairs[0]
    start_idx = text.find(start_marker)
    if start_idx == -1:
        print(f"[❗] Start marker tidak ditemukan di {kuartal}: {start_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    search_range = text[start_idx:]
    end_relative = search_range.find(end_marker)
    if end_relative == -1:
        print(f"[❗] End marker tidak ditemukan setelah start marker di {kuartal}: {end_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    end_idx = start_idx + end_relative

    orig_start_idx = text.lower().find(start_marker.lower())
    orig_end_idx = text.lower().find(end_marker.lower(), orig_start_idx)

    if orig_start_idx != -1 and orig_end_idx != -1:
        content_raw = text[orig_start_idx + len(start_marker): orig_end_idx]
        snippet = content_raw.strip()
    else:
        snippet = text[start_idx + len(start_marker): end_idx].strip()

    return snippet  # Kembalikan string langsung, bukan list atau dict

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, Optional

@dataclass
class Company:
    kuartal: str = "-"
    perusahaan: str = "-"
    satuan: str = "-"
    risiko_nilai_tukar: str = "-"


# Mapping: atribut class -> nama key di dictionary input
field_map: Dict[str, str] = {
    "kuartal": "Kuartal",
    "perusahaan": "Perusahaan",
    "satuan": "Disajikan dalam",
    "risiko_nilai_tukar": "Efek Risiko Nilai Tukar",
    "risiko_suku_bunga": "Efek Risiko Suku Bunga",
}

def get_str(value) -> str:
    if value is None:
        return "-"
    val = str(value).strip()
    return val if val else "-"

def generate_company(doc: Optional[dict]) -> Company:
    data = {}
    for attr, source_key in field_map.items():
        data[attr] = get_str(doc.get(source_key)) if doc else "-"
    return Company(**data)

import json

company = Company()

In [ ]:
def find_nilai_tukar(text, marker_pairs, company: Company, kuartal: str):
    # Gabungkan newline jadi spasi agar regex lebih fleksibel
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)

    # Cek apakah teks_kotor adalah list, dan gabungkan menjadi string jika iya
    if isinstance(teks_kotor, list):
        teks_kotor = " ".join(teks_kotor)

    teks_bersih = teks_kotor.replace("\n", " ")

    company.risiko_nilai_tukar = teks_bersih

def find_suku_bunga(text, marker_pairs, company: Company, kuartal: str):
    # Gabungkan newline jadi spasi agar regex lebih fleksibel
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)

    # Cek apakah teks_kotor adalah list, dan gabungkan menjadi string jika iya
    if isinstance(teks_kotor, list):
        teks_kotor = " ".join(teks_kotor)

    teks_bersih = teks_kotor.replace("\n", " ")

    company.risiko_suku_bunga = teks_bersih

In [ ]:
def process_all_markers(text, kuartal, emiten):
    company = Company()
    company.perusahaan = emiten
    company.kuartal = kuartal

    markers = marker_config_map.get(emiten.upper())

    for marker_name, marker_pairs in markers.items():
        function_name = marker_to_function.get(marker_name)
        if not function_name:
            continue

        func = globals().get(function_name)
        if not func:
            print(f"[⚠️] Fungsi '{function_name}' tidak ditemukan.")
            continue

        # ⬇️ Fungsi sekarang juga menerima objek company yang akan di-update
        func(text, marker_pairs, company, kuartal)

    return company

# Mapping marker ke fungsi
marker_to_function = {
    "marker1": "find_satuan",
    "marker2": "find_nilai_tukar",
    "marker3": "find_suku_bunga"
}

def find_satuan(text, marker_pairs, company: Company, kuartal: str):
    results = find_paragraphs_by_marker_pairs(text, marker_pairs,kuartal)
    # satuan = results if results else "-"
    company.satuan = results

In [ ]:
# Fungsi pemrosesan individual
def process_file(emiten, year, quarter):
    kuartal_label = f"{year}-{quarter}"
    pdf_file = f"{base_path}/{emiten}/{emiten}-{year}-{quarter}.pdf"
    if not os.path.exists(pdf_file):
        print(f"[⚠️] File tidak ditemukan: {pdf_file}")
        return None

    print(f"\n🔍 Memproses: {emiten} - {kuartal_label}")
    try:
        start_time = time.time()
        text = extract_text(pdf_file)
        hasil = process_all_markers(text, kuartal_label, emiten)
        elapsed = time.time() - start_time
        print(f"\n✅ {kuartal_label}-{emiten} berhasil diproses dalam {elapsed:.2f} detik.")
        return f"{emiten}-{kuartal_label}", hasil.__dict__
    except Exception as e:
        print(f"[⚠️] Error pada {emiten}-{kuartal_label}: {e}")
        return None

In [ ]:
# Konfigurasi
output_json_file = "hasil_semua.json"
output_file = "hasil_ekstraksi_kurs_bunga.xlsx"
benchmark_file = "benchmark_waktu.xlsx"

# Muat hasil sebelumnya jika ada
if os.path.exists(output_json_file):
    with open(output_json_file, "r", encoding="utf-8") as f:
        all_results_dicts_dinamis = json.load(f)
else:
    all_results_dicts_dinamis = {}

# Simpan waktu proses per kuartal
benchmark_data = []

# Cek apakah sudah diproses
def is_processed(emiten, year, quarter):
    key = f"{emiten}-{year}-{quarter}"
    return key in all_results_dicts_dinamis

# Proses satu batch
def process_batch(batch_emiten, year, quarter):
    batch_results = []
    for emiten in batch_emiten:
        if not is_processed(emiten, year, quarter):
            start_time = time.time()
            result = process_file(emiten, year, quarter)
            if result:
                key, data = result
                batch_results.append((key, data, time.time() - start_time))
    return batch_results

# Bagi jadi batch
emiten_batches = [all_companies[i:i + batch_size] for i in range(0, len(all_companies), batch_size)]
# Jalankan per batch per kuartal
for batch_index, batch_emiten in enumerate(emiten_batches, start=1):
    print(f"\n🚀 Mulai Batch {batch_index}: {batch_emiten}")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_batch, batch_emiten, year, quarter)
            for year in years
            for quarter in quarters
        ]

        for future in as_completed(futures):
            batch_results = future.result()
            for key, data, durasi in batch_results:
                all_results_dicts_dinamis[key] = data
                emiten, year, quarter = key.split("-")
                benchmark_data.append({
                    "Emiten": emiten,
                    "Kuartal": f"{year}-{quarter}",
                    "Durasi (detik)": round(durasi, 2)
                })

                # Simpan progres JSON setiap selesai 1 kuartal
                with open(output_json_file, "w", encoding="utf-8") as f:
                    json.dump(all_results_dicts_dinamis, f, indent=2, ensure_ascii=False)

# Simpan ke Excel
df = pd.DataFrame(list(all_results_dicts_dinamis.values()))
if 'Emiten' in df.columns and 'Kuartal' in df.columns:
    df.sort_values(by=["Emiten", "Kuartal"], inplace=True)

# Urutkan benchmark berdasarkan 'Emiten' dan 'Kuartal'
benchmark_df = pd.DataFrame(benchmark_data)
benchmark_df.sort_values(by=["Emiten", "Kuartal"], inplace=True)

df.to_excel(output_file, index=False)
print(f"\n✅ File hasil utama: {output_file}")
benchmark_df.to_excel(benchmark_file, index=False)


print(f"⏱️ File benchmark waktu dibuat: {benchmark_file}")

# Waktu total program
end_program_time = time.time()
print(f"\n🕒 Total waktu runtime program: {round(end_program_time - start_program_time, 2)} detik")

# Auto-download jika di Colab
try:
    from google.colab import files
    files.download(output_file)
    files.download(benchmark_file)
except:
    pass


🚀 Mulai Batch 1: ['TLKM', 'INDF', 'INKP', 'ADRO', 'UNTR', 'JSMR', 'ICBP', 'MEDC', 'ISAT', 'GIAA']
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/INDF/INDF-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/INKP/INKP-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/ADRO/ADRO-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/JSMR/JSMR-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/ICBP/ICBP-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/MEDC/MEDC-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/ISAT/ISAT-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/GIAA/GIAA-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/ADRO/ADRO-2023-Q3.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/TLKM/TLKM-2024-Q1.pdf
[

IllegalCharacterError: Grup  menghadapi risiko nilai tukar mata uang asing  karena pinjaman, penjualan dan pembelian  dalam mata uang dolar Amerika Serikat atau  harga yang secara signifikan dipengaruhi oleh  tolak ukur perubahan harganya dalam mata  uang asing (terutama dolar AS) seperti yang  dikutip dari pasar internasional. Grup tidak  mempunyai kebijakan lindung nilai yang formal  untuk laju pertukaran mata uang asing, namun  Grup memiliki fasilitas dari bank tertentu untuk  mengadakan transaksi lindung nilai.     7 K H   * U R X S ¶ V   U H S R U W L Q J   F X U The Group faces foreign exchange risk as its  borrowings, sales and purchases are either  denominated in U.S dollar or whose price is  significantly influenced by their benchmark  price movements in foreign currencies (mainly  U.S. dollar) as quoted in the international  markets. The Group does not have any formal  hedging policy for foreign exchange exposure,  but it has a facility from a certain bank to enter  into h

In [ ]:
import re

def clean_text(text):
    # Remove control characters and other problematic characters
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', text)
    return text

# ... (Your existing code) ...

# Simpan ke Excel
df = pd.DataFrame(list(all_results_dicts_dinamis.values()))
# Clean text in specific columns before saving to Excel
for col in ['risiko_nilai_tukar', 'risiko_suku_bunga']:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

if 'Emiten' in df.columns and 'Kuartal' in df.columns:
    df.sort_values(by=["Emiten", "Kuartal"], inplace=True)

df.to_excel(output_file, index=False)
print(f"\n✅ File hasil utama: {output_file}")


✅ File hasil utama: hasil_ekstraksi_kurs_bunga.xlsx


In [ ]:
print(f"\n✅ File hasil utama: {output_file}")
benchmark_df.to_excel(benchmark_file, index=False)


print(f"⏱️ File benchmark waktu dibuat: {benchmark_file}")

# Waktu total program
end_program_time = time.time()
print(f"\n🕒 Total waktu runtime program: {round(end_program_time - start_program_time, 2)} detik")

# Auto-download jika di Colab
try:
    from google.colab import files
    files.download(output_file)
    files.download(benchmark_file)
except:
    pass


✅ File hasil utama: hasil_ekstraksi_kurs_bunga.xlsx
⏱️ File benchmark waktu dibuat: benchmark_waktu.xlsx

🕒 Total waktu runtime program: 2231.01 detik


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>